# Function Call == $\frac{1}{2}$ context switch

* Done with call instruction
    * `call foo`
    * `call %eax`
* Call instruction
    * Address of next instr (ret addr) pushed onto stack
    * Load EIP with call instr arg (causes a jump to take place)
* Last instruction of function call is `ret`
    * Pop top value from stack (retaddr) to EIP
    * New value in EIP causes return
* For this to work, stack and register contents must be in the same state as before the call, except for any changes that the caller+callee agree upon

## Anatomy of Function Call
**Caller**
* Push args on stack in reverse order
* Execute call instruction to jump to subroutine (causes addr of next instruction to be pushed onto the stack)

**Callee**
* Save `ebp` to stack
* Copy current stack pointer to frame pointer
* Save registers that will get clobbered
* Caller assumes `eax` will be clobbered
* Adjust `esp` to make space for local variables


* `%ebp` is frame pointer
* `%esp` is stack pointer


# Calling Conventions
* Defines reg and stack usage on function calls
* This permits compiler to compile code
* Conventions are
    * Compiler
    * Architecture
    * OS dependent
* Our system is
    * 32bit, x86, gcc
    * calling convention is cdecl

## cdecl
* Params on stack, pushed in reverse order
* eax, ecx, edx can be overwritten by callee
* return value in eax
* ebp (frame ptr) pushed by callee on function entry
* ebp then points to top of the stack and params and local vars

### Callee Side
```asm
foo:
    pushl %ebp
    movl %esp, %ebp
    movl 12(%ebp), %eax
    addl 8(%ebp), %eax
    leave
    ret
```

### Upon return
```asm
addl $8, %esp // This "deletes" the old stack variables by moving the pointer behind them
movl %eax, -12(%ebp)
```

### Callstack Convention
* **lowaddr**
* working space (saved registers)
* local vars
* saved EBP
* RET ADDR (pushed by call function)
* params
* **highaddr**

# Process Switching
A context switch passes control from
* running process --> kernel --> selected process
* Mecahnism that causes switch to take place is not a function call but an "*interrupt*"
* Unlike a function call, register values must be preserved
    * exception: syscall where `eax` gets the return value
    * since this is like an interrupt, we have to assume all registers were being used. We need to save EVERYTHING
    
Since processed being switched from didn't make a call....
* kernel must save all of the process's state
* kernel must recover its state when its done

When returning to the process
* context switcher must save all kernel's registers
* context switcher must provide process with all the register values it had previous saved for that process

Note we switched stacks!
* Kernel has to remember its stack pointer so that it can be recovered
* Saving the stack pointer can't be done with a local variable, since we switched stacks!
    * We need to use static/global variables! These are stored in the `data` segment of a process's memory

## Anatomy of Exception/Interrupt
* Int occurs
* CPU actions
    * push EFLAGS
    * push CS reg
    * push EIP
    * push error code
    * Loads new CS and EIP from IDT, and cause jump to ISR
* switch the kernel mode

To go back...
* `iret`
    * reload EIP
    * relad CS
    * reload EFLAGS
* switch to user mode

So our process switching has to make it seem like an interrupt occurred. Our stack needs to look exactly as above for `iret`

## ISR
* Begins while still using process's stack
* Save process's context and stack pointer
* Switch to kernel stack
* Execute service code (kernel)
* Select process to execute
* Save kernel context and stack pointer
* Switch the selected process

### Tricks
* Make it look like the kernel is calling a process
* Make it look like the process is returning to kernel code

---

*October 5, 2016*

---

# Actually Switching Between Process and Kernel

### Implications
* When create the process, we have to synthesize an initial stack to make it look like it was interrupted (prior to even starting)
* Must make the stack look like the process was interrupted
* Consequently we need to store ptr to top of process's stack in PCB

# Creating a Process
* Acquire a PCB
* Initialize PCB
    * Setup addrspace (we can skip this in asst1)
    * Allocate physical memory to the process (just the stack in asst1)
    * Load code into addrspace (already done in assignment)
* **Initialize process context**
* Add to ready queue

## Context Initialization
* A process will be passed to the context switcher to start it executing
* Recall that this means we need to initialize the stack to appear as if a process was already running

### Positioning the Context
* Point to the end of allocated memory chunk
* Subtract a safety margin **and** size of context frame (4096 * 2 = 8kB)
* Stack pointer starts here
* Store this value as stack pointer value in PCB
* Fill in the context frame
    * edi to eax can be 0
    * iret_eip = first instruction of the function you create
    * iret_cs = get_cs()
    * eflags
        * iterrupt enable flag needs to be 0
        * **in fact, everything might as well just be 0 for asst1**
* Don't forget that the **context needs to be placed at the high address of malloc'd memory**


# Syscalls